# Kaggle Project

## Describe Your Dataset

**URL:** https://www.kaggle.com/datasets/lantian773030/pokemonclassification

**Task:**

#포켓몬 149마리의 이미지 데이터 셋을 흑백 이미지로 변환하여 분류(기존 dataset에서 Alolan Sandslash의 폴더는 삭제)
#dataset의 class가 많고 image data갯수가 적기 때문에 효율적인 학습을 위하여 transform을 통한 image data변형 및 gray scale로의 변형을 통해 학습의 효율을 높임
#class간 data 갯수가 불균형하므로 그에 따른 weight를 Loss 계산에 부과해 줌으로써 학습 효율을 높임
#단순 MLP(Multi-Layer Perceptron) 방식과 이미지 train에 쓰이는 CNN(Convolutional Neural Network) 방식 두가지 방식으로 학습을 진행

**Datasets**

* Train dataset: 전체 Dataset의 80%를 random split

* Validation dataset: 전체 Dataset의 10%

* Test dataset: 전체 Dataset의 10%

**Features(x):**

64x64 Grayscale Image data 

#### **Target(y):**

149 class

---

## Build Your Model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torch.utils.data import Dataset
from collections import Counter
import wandb
import random
import math
from torchvision import datasets, models, transforms
wandb.login()

wandb: Currently logged in as: sungwoobaek. Use `wandb login --relogin` to force relogin


True

### Data preprocessing

In [2]:
path = 'Midtermdata'
data_transforms = transforms.Compose([
    transforms.Resize(size=(64,64), antialias=True), # 이미지의 사이즈를 64x64의 사이즈로 Resize
    transforms.Grayscale(num_output_channels=1), # 컬러이미지를 흑백 이미지로 변환
    transforms.RandomHorizontalFlip(), # 좌우 반전 default 50%확률로 반전
    transforms.RandomVerticalFlip(), # 상하 반전 default 50%확률로 반전
    transforms.AutoAugment(policy=transforms.autoaugment.AutoAugmentPolicy.IMAGENET, interpolation=transforms.InterpolationMode.BILINEAR),
    # pytorch 내부적으로 정의하고 있는 policy를 사용하여 자동으로 augmentation
    transforms.ToTensor(), # Tensor 형태로 변환
    transforms.Normalize([0.5],[0.5]) # 정규화 
])
image_datasets = datasets.ImageFolder(path,data_transforms) 
train_size = int(0.8 * len(image_datasets)) # train data의 size(총 data의 80%)
validation_size = int(0.1 * len(image_datasets)) # validation data의 size(총 data의 10%)
test_size = len(image_datasets) - train_size - validation_size # test data의 size
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(image_datasets, [train_size, validation_size, test_size])
# image data set을 random하게 train, validation, test dataset으로 split
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=16,shuffle = True, drop_last=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=4,shuffle = True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=image_datasets, batch_size=4,shuffle = True, drop_last=True)
class_counts = Counter(train_dataset.dataset.targets)
train_weights = 1. / torch.tensor(list(class_counts.values()), dtype=torch.float).cuda() # class별 불균형을 맞추기 위한 train weight
class_counts = Counter(validation_dataset.dataset.targets)
validation_weights = 1. / torch.tensor(list(class_counts.values()), dtype=torch.float).cuda() # class별 불균형을 맞추기 위한 validation weight
class_counts = Counter(test_dataset.dataset.targets)
test_weights = 1. / torch.tensor(list(class_counts.values()), dtype=torch.float).cuda() # class별 불균형을 맞추기 위한 test weight

### Model Construction

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(in_features=64*64,out_features=512,bias=True),
            nn.ReLU(),
            nn.Linear(in_features=512,out_features=256,bias=True),
            nn.ReLU(),
            nn.Linear(in_features=256,out_features=149,bias=True),
        )
    def forward(self, x):
        x = self.fc_layer(x)
        return x

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(in_features=256, out_features=128, bias=True),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(in_features=128, out_features=149,bias=True),
            #nn.Softmax(dim=1)   
        )
    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.shape[0],-1)
        x = self.fc_layer(x)
        return x

mlp = MLP().cuda()
cnn = CNN().cuda()
train_criterion = torch.nn.CrossEntropyLoss(weight=train_weights)
validation_criterion = torch.nn.CrossEntropyLoss(weight=validation_weights)
test_criterion = torch.nn.CrossEntropyLoss(weight=test_weights)
mlp_optimizer = optim.Adagrad(mlp.parameters(), lr=0.01)
cnn_optimizer = optim.Adagrad(cnn.parameters(), lr=0.01)

In [4]:
def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)
wandb.init(
    project="Midterm_project",    
)
wandb.run.name = 'MLP'
wandb.run.save()
n_steps_per_epoch = math.ceil(len(train_loader.dataset) /16)
example_ct = 0
step_ct = 0

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


### Train Model & Select Model

In [5]:
for epoch in range(80):
    mlp.train()
    for index, (data, target) in enumerate(train_loader):
        X = data.view(-1,64*64).cuda()
        Y = target.cuda()
        output = mlp(X)
        loss = train_criterion(output, Y)

        mlp_optimizer.zero_grad()
        loss.backward()
        mlp_optimizer.step()

        example_ct += len(data)
        metrics = {"train/mlp_train_loss": loss,
                   "train/mlp_iter": (index + 1 + (n_steps_per_epoch * epoch)),
                   "train/mlp_example_ct":example_ct}
        if index + 1 < n_steps_per_epoch:
            wandb.log(metrics)

        step_ct +=1

        if (index+1) % 50 == 0:
            print("loss of {} epoch, {} index : {}".format(epoch+1, index+1, loss.item()))
        
    mlp.eval()
    with torch.no_grad():
        valid_loss = sum(validation_criterion(mlp(X.view(-1,64*64).cuda()), Y.cuda()) for X, Y in validation_loader)
        metrics = {"valid/mlp_valid_loss": valid_loss/len(validation_loader),
                   "valid/mlp_epoch": epoch+1}
        wandb.log(metrics)
    print("epoch", epoch+1,": ", valid_loss.item() / len(validation_loader))
    clear_output(wait=True)

loss of 80 epoch, 50 index : 3.2046921253204346
loss of 80 epoch, 100 index : 4.012154579162598
loss of 80 epoch, 150 index : 3.847484588623047
loss of 80 epoch, 200 index : 3.754336357116699
loss of 80 epoch, 250 index : 3.688281774520874
loss of 80 epoch, 300 index : 3.582340955734253
loss of 80 epoch, 350 index : 3.8492190837860107
loss of 80 epoch, 400 index : 3.5572404861450195
loss of 80 epoch, 450 index : 5.179178714752197
loss of 80 epoch, 500 index : 4.005454063415527
loss of 80 epoch, 550 index : 2.923060894012451
loss of 80 epoch, 600 index : 3.322357177734375
loss of 80 epoch, 650 index : 3.510493040084839
epoch 80 :  4.831412953032545


In [6]:
wandb.run.name = 'CNN'
wandb.run.save()

True

In [7]:
n_steps_per_epoch = math.ceil(len(train_loader.dataset) / 16)
example_ct = 0
step_ct = 0
for epoch in range(1000):
    cnn.train()
    for index, (data, target) in enumerate(train_loader):
        X = data.cuda()
        Y = target.cuda()
        
        output = cnn(X)

        loss = train_criterion(output, Y)

        cnn_optimizer.zero_grad()
        loss.backward()
        cnn_optimizer.step()

        example_ct += len(data)
        metrics = {"train/cnn_train_loss": loss,
                   "train/cnn_iter": (index + 1 + (n_steps_per_epoch * epoch)),
                   "train/cnn_example_ct":example_ct}
        if index + 1 < n_steps_per_epoch:
            wandb.log(metrics)

        step_ct +=1

        if (index+1) % 50 == 0:
            print("loss of {} epoch, {} index : {}".format(epoch+1, index+1, loss.item()))
        
    cnn.eval()
    with torch.no_grad():
        valid_loss = sum(validation_criterion(cnn(X.cuda()), Y.cuda()) for X, Y in validation_loader)
        metrics = {"valid/cnn_valid_loss": valid_loss.item() / len(validation_loader),
                   "valid/cnn_epoch": epoch+1}
        wandb.log(metrics)
    print("epoch", epoch+1,": ", valid_loss.item() / len(validation_loader))
    clear_output(wait=True)

loss of 3000 epoch, 50 index : 3.0963430404663086
loss of 3000 epoch, 100 index : 3.269336223602295
loss of 3000 epoch, 150 index : 2.1023497581481934
loss of 3000 epoch, 200 index : 3.4918558597564697
loss of 3000 epoch, 250 index : 3.0338401794433594
loss of 3000 epoch, 300 index : 3.351933717727661
loss of 3000 epoch, 350 index : 3.8153700828552246
loss of 3000 epoch, 400 index : 2.6145122051239014
loss of 3000 epoch, 450 index : 3.7367429733276367
loss of 3000 epoch, 500 index : 2.742234706878662
loss of 3000 epoch, 550 index : 2.9828402996063232
loss of 3000 epoch, 600 index : 2.8723769187927246
loss of 3000 epoch, 650 index : 2.9956963062286377
epoch 3000 :  3.685441056652182


---

## Performance

In [13]:
mlp.eval()  # test case 학습 방지를 위함
test_loss = 0
correct = 0
with torch.no_grad():
  for data, target in test_loader:
    data = data.view(-1,64*64).cuda()
    target = target.cuda()
    output = mlp(data)
    test_loss += test_criterion(output, target).item() # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Test set: Average loss: 13465.8672, Accuracy: 1208/13559 (9%)



In [11]:
cnn.eval()  # test case 학습 방지를 위함
test_loss = 0
correct = 0
with torch.no_grad():
  for data, target in test_loader:
    data = data.cuda()
    target = target.cuda()
    output = cnn(data)
    test_loss += test_criterion(output, target).item() # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Test set: Average loss: 10756.1345, Accuracy: 2254/13559 (17%)



In [15]:
torch.save(mlp.state_dict(), "pth/mlp.pth")
torch.save(cnn.state_dict(), "pth/cnn.pth")

The results explains

....